__Probing Language Models__

This notebook serves as a start for your NLP2 assignment on probing Language Models. This notebook will become part of the contents that you will submit at the end, so make sure to keep your code (somewhat) clean :-)

__note__: This is the first time _anyone_ is doing this assignment. That's exciting! But it might well be the case that certain aspects are too unclear. Do not hesitate at all to reach to me once you get stuck, I'd be grateful to help you out.

__note 2__: This assignment is not dependent on big fancy GPUs. I run all this stuff on my own 3 year old CPU, without any Colab hassle. So it's up to you to decide how you want to run it.

# Models

For the Transformer models you are advised to make use of the `transformers` library of Huggingface: https://github.com/huggingface/transformers
Their library is well documented, and they provide great tools to easily load in pre-trained models.

In [1]:
import torch
import numpy as np
import time

from collections import defaultdict
from typing import List
from conllu import parse_incr, TokenList
from torch import Tensor
from transformers import GPT2Model, GPT2Tokenizer

from lstm.model import RNNModel
CUTOFF = 30
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [2]:
transformer = GPT2Model.from_pretrained('distilgpt2', output_hidden_states=True)
print("Model ready")
tokenizer = GPT2Tokenizer.from_pretrained('distilgpt2')
print("Tokenizer ready")
# Note that some models don't return the hidden states by default.
# This can be configured by passing `output_hidden_states=True` to the `from_pretrained` method.

Model ready
Tokenizer ready


In [3]:
tokenizer.decode(tokenizer.encode("I like puffy cheeseballs"))

'I like puffy cheeseballs'

In [4]:
#
## Your code for initializing the rnn model(s)
#
# The Gulordava LSTM model can be found here: 
# https://drive.google.com/open?id=1w47WsZcZzPyBKDn83cMNd0Hb336e-_Sy
#
# N.B: I have altered the RNNModel code to only output the hidden states that you are interested in.
# If you want to do more experiments with this model you could have a look at the original code here:
# https://github.com/facebookresearch/colorlessgreenRNNs/blob/master/src/language_models/model.py
#
model_location = 'lstm/gulordava.pt'
lstm = RNNModel('LSTM', 50001, 650, 650, 2)
lstm.load_state_dict(torch.load(model_location))


# This LSTM does not use a Tokenizer like the Transformers, but a Vocab dictionary that maps a token to an id.
with open('lstm/vocab.txt') as f:
    w2i = {w.strip(): i for i, w in enumerate(f)}

vocab = defaultdict(lambda: w2i["<unk>"])
vocab.update(w2i)
i2w = { w2i[k]:k for k in w2i}

In [5]:
lstm.eval()
print(type(lstm))
with torch.no_grad():
    input_sent = "My sister is 20"
    inputs = torch.tensor([vocab[z] for z in input_sent.split()]).unsqueeze(0)
    outputs = lstm(inputs, lstm.init_hidden(1), True)
    argmaxes = torch.argmax(outputs, dim=1)
    last = argmaxes[-1]
    print("Voc size", outputs.shape[-1])
    print(input_sent, i2w[last.item()])

<class 'lstm.model.RNNModel'>
Voc size 50001
My sister is 20 years


It is a good idea that before you move on, you try to feed some text to your LMs; and check if everything works accordingly. 

# Data

For this assignment you will train your probes on __treebank__ corpora. A treebank is a corpus that has been *parsed*, and stored in a representation that allows the parse tree to be recovered. Next to a parse tree, treebanks also often contain information about part-of-speech tags, which is exactly what we are after now.

The treebank you will use for now is part of the Universal Dependencies project. I provide a sample of this treebank as well, so you can test your setup on that before moving on to larger amounts of data.

Make sure you accustom yourself to the format that is created by the `conllu` library that parses the treebank files before moving on. For example, make sure you understand how you can access the pos tag of a token, or how to cope with the tree structure that is formed using the `to_tree()` functionality.

# Generating Representations

We now have our data all set, our models are running and we are good to go!

The next step is now to create the model representations for the sentences in our corpora. Once we have generated these representations we can store them, and train additional diagnostic (/probing) classifiers on top of the representations.

There are a few things you should keep in mind here. Read these carefully, as these tips will save you a lot of time in your implementation.
- Transformer models make use of Byte-Pair Encodings (BPE), that chunk up a piece of next in subword pieces. For example, a word such as "largely" could be chunked up into "large" and "ly". We are interested in probing linguistic information on the __word__-level. Therefore, we will follow the suggestion of Hewitt et al. (2019a, footnote 4), and create the representation of a word by averaging over the representations of its subwords. So the representation of "largely" becomes the average of that of "large" and "ly".

- Subword chunks never overlap multiple tokens. In other words, say we have a phrase like "None of the", then the tokenizer might chunk that into "No"+"ne"+" of"+" the", but __not__ into "No"+"ne o"+"f the", as those chunks overlap multiple tokens. This is great for our setup! Otherwise it would have been quite challenging to distribute the representation of a subword over the 2 tokens it belongs to.

- If you closely examine the provided treebank, you will notice that some tokens are split up into multiple pieces, that each have their own POS-tag. For example, in the first sentence the word "Al-Zaman" is split into "Al", "-", and "Zaman". In such cases, the conllu `TokenList` format will add the following attribute: `('misc', OrderedDict([('SpaceAfter', 'No')]))` to these tokens. Your model's tokenizer does not need to adhere to the same tokenization. E.g., "Al-Zaman" could be split into "Al-"+"Za"+"man", making it hard to match the representations with their correct pos-tag. Therefore I recommend you to not tokenize your entire sentence at once, but to do this based on the chunking of the treebank. Make sure to still incoporate the spaces in a sentence though, as these are part of the BPE of the tokenizer. The tokenizer for GPT-2 adds spaces at the start of a token.

- The LSTM LM does not have the issues related to subwords, but is far more restricted in its vocabulary. Make sure you keep the above points in mind though, when creating the LSTM representations. You might want to write separate functions for the LSTM, but that is up to you.

I would like to stress that if you feel hindered in any way by the simple code structure that is presented here, you are free to modify it :-) Just make sure it is clear to an outsider what you're doing, some helpful comments never hurt.

In [19]:
from utils import assert_sen_reps, fetch_sen_reps, parse_corpus, bitch
assert_sen_reps(transformer, tokenizer, lstm, vocab)

Doing LSTM: False
Doing LSTM: True
torch.Size([29, 768]) torch.Size([29, 768])
All is well!


In [20]:
from utils import create_or_load_pos_data

train_x, train_y, vocab, words_train = create_or_load_pos_data("train", transformer, tokenizer, cutoff=CUTOFF)
dev_x, dev_y, vocab, words_dev = create_or_load_pos_data("dev", transformer, tokenizer, vocab, cutoff=CUTOFF)
test_x, test_y, vocab, words_test = create_or_load_pos_data("test", transformer, tokenizer, vocab, cutoff=CUTOFF)

In [21]:
#TODO: MAKE THIS SHIZZ WORK BETTER BECAUSE SPAGHETTI^2
flatten_train = [word for sublist in words_train for word in sublist]
flatten_dev   = [word for sublist in words_dev for word in sublist]
flatten_test  = [word for sublist in words_test for word in sublist]

In [22]:
print(len(flatten_train))

667


# Diagnostic Classification

We now have our models, our data, _and_ our representations all set! Hurray, well done. We can finally move onto the cool stuff, i.e. training the diagnostic classifiers (DCs).

DCs are simple in their complexity on purpose. To read more about why this is the case you could already have a look at the "Designing and Interpreting Probes with Control Tasks" by Hewitt and Liang (esp. Sec. 3.2).

A simple linear classifier will suffice for now, don't bother with adding fancy non-linearities to it.

I am personally a fan of the `skorch` library, that provides `sklearn`-like functionalities for training `torch` models, but you are free to train your dc using whatever method you prefer.

As this is an Artificial Intelligence master and you have all done ML1 + DL, I expect you to use your train/dev/test splits correctly ;-)

In [23]:
from datasets import POSDataset, find_distribution
import torch.utils.data as data
    
train_loader = data.DataLoader(POSDataset(train_x, train_y), batch_size=16)
dev_loader = data.DataLoader(POSDataset(dev_x, dev_y), batch_size=16)
test_loader = data.DataLoader(POSDataset(test_x, test_y), batch_size=16)

In [24]:
dist = find_distribution(data.DataLoader(POSDataset(train_x, train_y), batch_size=1))
print(dist)

defaultdict(<function find_distribution.<locals>.<lambda> at 0x7f1574aaf040>, {0: 89, 1: 66, 2: 36, 3: 114, 4: 58, 5: 75, 6: 83, 7: 42, 8: 33, 9: 6, 10: 16, 11: 15, 12: 21, 13: 13})


In [25]:
# DIAGNOSTIC CLASSIFIER
import torch.nn as nn
class POSProbe(nn.Module):
    def __init__(self, repr_size, pos_size):
        super().__init__()
        self.linear = nn.Linear(repr_size, pos_size)
        
    def forward(self, x):
        return self.linear(x)
    
def eval_given_dataloader(loader, model):
    model.eval()
    correct = 0.0
    total = 0.0
    for x,y in loader:
        x = x.to(device)
        y = y.to(device)
        outputs = model(x)
        preds = torch.argmax(outputs,dim=1)
        c = torch.sum(torch.eq(preds, y))
        correct += c.item()
        total += y.shape[0]
    return correct/total
    
def train(my_model, train_loader, dev_loader, epoch_amount = 10):
    ce = nn.CrossEntropyLoss()
    optim = torch.optim.Adam(my_model.parameters())
    for i in range(epoch_amount):
        my_model.train()
        epoch_correct = 0.0
        epoch_total = 0.0
        for x,y in train_loader:
            
            x = x.to(device)
            y = y.to(device)
            outputs = my_model(x)
            preds = torch.argmax(outputs,dim=1)
            correct = torch.sum(torch.eq(preds, y))
            accuracy = correct.item()/y.shape[0]
            loss = ce(outputs, y)

            optim.zero_grad()
            loss.backward()
            optim.step()
            
            epoch_correct += correct.item()
            epoch_total += y.shape[0]
        print("Epoch",i,"accuracy", epoch_correct/epoch_total, eval_given_dataloader(dev_loader, my_model))
    
        
model = POSProbe(768, len(dist)).to(device)
train(model, train_loader, dev_loader, 10)
print("Test accuracy", eval_given_dataloader(test_loader, model))

Epoch 0 accuracy 0.3793103448275862 0.505730659025788
Epoch 1 accuracy 0.7496251874062968 0.6361031518624641
Epoch 2 accuracy 0.8470764617691154 0.6876790830945558
Epoch 3 accuracy 0.889055472263868 0.7177650429799427
Epoch 4 accuracy 0.9160419790104948 0.7292263610315186
Epoch 5 accuracy 0.9295352323838081 0.7406876790830945
Epoch 6 accuracy 0.9445277361319341 0.7521489971346705
Epoch 7 accuracy 0.9505247376311844 0.7593123209169055
Epoch 8 accuracy 0.952023988005997 0.7621776504297995
Epoch 9 accuracy 0.9610194902548725 0.7664756446991404
Test accuracy 0.7245508982035929


# Trees

For our gold labels, we need to recover the node distances from our parse tree. For this we will use the functionality provided by `ete3`, that allows us to compute that directly. I have provided code that transforms a `TokenTree` to a `Tree` in `ete3` format.

In [13]:
from tree_utils import * 

In [14]:
corpus = parse_corpus('data/sample/en_ewt-ud-train.conllu')
item = corpus[0]
print(corpus[0])
tokentree = item.to_tree()
ete3_tree = tokentree_to_ete(tokentree)
print(ete3_tree)

TokenList<Al, -, Zaman, :, American, forces, killed, Shaikh, Abdullah, al, -, Ani, ,, the, preacher, at, the, mosque, in, the, town, of, Qaim, ,, near, the, Syrian, border, .>

   /-2
  |
  |--3
  |
  |--4
  |
  |   /6 /-5
  |  |
  |  |   /-9
  |  |  |
  |  |  |--10
  |  |  |
  |  |  |--11
  |  |-8|
  |  |  |--12
  |-7|  |
  |  |  |--13
  |  |  |
  |  |   \15/-14
-1|  |
  |  |   /-16
  |  |  |
  |  |  |--17
  |  |  |
  |   \18   /-19
  |     |  |
  |     |  |--20
  |     |  |
  |     |  |-23/-22
  |      \21
  |        |--24
  |        |
  |        |   /-25
  |        |  |
  |         \28--26
  |           |
  |            \-27
  |
   \-29


As you can see we label a token by its token id (converted to a string). Based on these id's we are going to retrieve the node distances.

To create the true distances of a parse tree in our treebank, we are going to use the `.get_distance` method that is provided by `ete3`: http://etetoolkit.org/docs/latest/tutorial/tutorial_trees.html#working-with-branch-distances

We will store all these distances in a `torch.Tensor`.

Please fill in the gap in the following method. I recommend you to have a good look at Hewitt's blog post  about these node distances.

In [15]:
all_distances = create_gold_distances(corpus[:10])
all_distances[0]

tensor([[0., 1., 1., 1., 1., 1., 2., 2., 2., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
         3., 4., 4., 4., 4., 4., 4., 5., 5., 5., 5.],
        [1., 0., 2., 2., 2., 2., 3., 3., 3., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
         4., 5., 5., 5., 5., 5., 5., 6., 6., 6., 6.],
        [1., 2., 0., 2., 2., 2., 3., 3., 3., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
         4., 5., 5., 5., 5., 5., 5., 6., 6., 6., 6.],
        [1., 2., 2., 0., 2., 2., 3., 3., 3., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
         4., 5., 5., 5., 5., 5., 5., 6., 6., 6., 6.],
        [1., 2., 2., 2., 0., 2., 1., 1., 1., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
         2., 3., 3., 3., 3., 3., 3., 4., 4., 4., 4.],
        [1., 2., 2., 2., 2., 0., 3., 3., 3., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
         4., 5., 5., 5., 5., 5., 5., 6., 6., 6., 6.],
        [2., 3., 3., 3., 1., 3., 0., 2., 2., 1., 3., 3., 3., 3., 3., 3., 3., 3.,
         3., 4., 4., 4., 4., 4., 4., 5., 5., 5., 5.],
        [2., 3., 3., 3., 1., 3., 2., 0., 2., 3., 1., 1.

The next step is now to do the previous step the other way around. After all, we are mainly interested in predicting the node distances of a sentence, in order to recreate the corresponding parse tree.

Hewitt et al. reconstruct a parse tree based on a _minimum spanning tree_ (MST, https://en.wikipedia.org/wiki/Minimum_spanning_tree). Fortunately for us, we can simply import a method from `scipy` that retrieves this MST.

Let's have a look at what this looks like, by looking at a relatively short sentence in the sample corpus.

If your addition to the `create_gold_distances` method has been correct, you should be able to run the following snippet. This then shows you the original parse tree, the distances between the nodes, and the MST that is retrieved from these distances. Can you spot the edges in the MST matrix that correspond to the edges in the parse tree?

In [16]:
item = corpus[5]
tokentree = item.to_tree()
ete3_tree = tokentree_to_ete(tokentree)
print(ete3_tree, '\n')

gold_distance = create_gold_distances(corpus[5:6])[0]
print(gold_distance, '\n')

mst = create_mst(gold_distance)
print(mst)


   /2 /-1
  |
  |--3
  |
  |--4
  |
  |   /-6
  |  |
-5|  |--7
  |-8|
  |  |   /-9
  |  |  |
  |   \12--10
  |     |
  |      \-11
  |
   \-13 

tensor([[0., 1., 1., 1., 1., 1., 2., 2., 2., 2., 3., 3., 3.],
        [1., 0., 2., 2., 2., 2., 1., 3., 3., 3., 4., 4., 4.],
        [1., 2., 0., 2., 2., 2., 3., 3., 3., 3., 4., 4., 4.],
        [1., 2., 2., 0., 2., 2., 3., 3., 3., 3., 4., 4., 4.],
        [1., 2., 2., 2., 0., 2., 3., 1., 1., 1., 2., 2., 2.],
        [1., 2., 2., 2., 2., 0., 3., 3., 3., 3., 4., 4., 4.],
        [2., 1., 3., 3., 3., 3., 0., 4., 4., 4., 5., 5., 5.],
        [2., 3., 3., 3., 1., 3., 4., 0., 2., 2., 3., 3., 3.],
        [2., 3., 3., 3., 1., 3., 4., 2., 0., 2., 3., 3., 3.],
        [2., 3., 3., 3., 1., 3., 4., 2., 2., 0., 1., 1., 1.],
        [3., 4., 4., 4., 2., 4., 5., 3., 3., 1., 0., 2., 2.],
        [3., 4., 4., 4., 2., 4., 5., 3., 3., 1., 2., 0., 2.],
        [3., 4., 4., 4., 2., 4., 5., 3., 3., 1., 2., 2., 0.]]) 

[[0. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0

Now that we are able to map edge distances back to parse trees, we can create code for our quantitative evaluation. For this we will use the Undirected Unlabeled Attachment Score (UUAS), which is expressed as:

$$\frac{\text{number of predicted edges that are an edge in the gold parse tree}}{\text{number of edges in the gold parse tree}}$$

To do this, we will need to obtain all the edges from our MST matrix. Note that, since we are using undirected trees, that an edge can be expressed in 2 ways: an edge between node $i$ and node $j$ is denoted by both `mst[i,j] = 1`, or `mst[j,i] = 1`.

You will write code that computes the UUAS score for a matrix of predicted distances, and the corresponding gold distances. I recommend you to split this up into 2 methods: 1 that retrieves the edges that are present in an MST matrix, and one general method that computes the UUAS score.

In [17]:
mst_edges = edges(mst)
assert (0,1) in mst_edges

## Structural Probes

We now have everything in place to start doing the actual exciting stuff: training our structural probe!
    
To make life easier for you, we will simply take the `torch` code for this probe from John Hewitt's repository. This allows you to focus on the training regime from now on.

In [6]:
import torch.nn as nn
import torch


class StructuralProbe(nn.Module):
    """ Computes squared L2 distance after projection by a matrix.
    For a batch of sentences, computes all n^2 pairs of distances
    for each sentence in the batch.
    """
    def __init__(self, model_dim, rank, device="cpu"):
        super().__init__()
        self.probe_rank = rank
        self.model_dim = model_dim
        
        self.proj = nn.Parameter(data = torch.zeros(self.model_dim, self.probe_rank))
        
        nn.init.uniform_(self.proj, -0.05, 0.05)
        self.to(device)

    def forward(self, batch):
        """ Computes all n^2 pairs of distances after projection
        for each sentence in a batch.
        Note that due to padding, some distances will be non-zero for pads.
        Computes (B(h_i-h_j))^T(B(h_i-h_j)) for all i,j
        Args:
          batch: a batch of word representations of the shape
            (batch_size, max_seq_len, representation_dim)
        Returns:
          A tensor of distances of shape (batch_size, max_seq_len, max_seq_len)
        """
        transformed = torch.matmul(batch, self.proj)
        
        batchlen, seqlen, rank = transformed.size()
        
        transformed = transformed.unsqueeze(2)
        transformed = transformed.expand(-1, -1, seqlen, -1)
        transposed = transformed.transpose(1,2)
        
        diffs = transformed - transposed
        
        squared_diffs = diffs.pow(2)
        squared_distances = torch.sum(squared_diffs, -1)

        return squared_distances

    
class L1DistanceLoss(nn.Module):
    """Custom L1 loss for distance matrices."""
    def __init__(self):
        super().__init__()

    def forward(self, predictions, label_batch, length_batch):
        """ Computes L1 loss on distance matrices.
        Ignores all entries where label_batch=-1
        Normalizes first within sentences (by dividing by the square of the sentence length)
        and then across the batch.
        Args:
          predictions: A pytorch batch of predicted distances
          label_batch: A pytorch batch of true distances
          length_batch: A pytorch batch of sentence lengths
        Returns:
          A tuple of:
            batch_loss: average loss in the batch
            total_sents: number of sentences in the batch
        """
        labels_1s = (label_batch != -1).float()
        predictions_masked = predictions * labels_1s
        labels_masked = label_batch * labels_1s
        total_sents = torch.sum((length_batch != 0)).float()
        squared_lengths = length_batch.pow(2).float()

        if total_sents > 0:
            loss_per_sent = torch.sum(torch.abs(predictions_masked - labels_masked), dim=(1,2))
            normalized_loss_per_sent = loss_per_sent / squared_lengths
            batch_loss = torch.sum(normalized_loss_per_sent) / total_sents
        
        else:
            batch_loss = torch.tensor(0.0)
        
        return batch_loss, total_sents


I have provided a rough outline for the training regime that you can use. Note that the hyper parameters that I provide here only serve as an indication, but should be (briefly) explored by yourself.

As can be seen in Hewitt's code above, there exists functionality in the probe to deal with batched input. It is up to you to use that: a (less efficient) method can still incorporate batches by doing multiple forward passes for a batch and computing the backward pass only once for the summed losses of all these forward passes. (_I know, this is not the way to go, but in the interest of time that is allowed ;-), the purpose of the assignment is writing a good paper after all_).

In [11]:
import importlib
import tree_utils
importlib.reload(tree_utils)
from tree_utils import create_or_load_structural_data
trainxy = create_or_load_structural_data("train", transformer, tokenizer, cutoff=CUTOFF)
devxy = create_or_load_structural_data("dev", transformer, tokenizer, cutoff=CUTOFF)
testxy = create_or_load_structural_data("test", transformer, tokenizer, cutoff=CUTOFF)

In [12]:
from datasets import StructuralDataset, pad_batch
import torch.utils.data as data

batch_size = 32
train_loader = data.DataLoader(StructuralDataset(*trainxy), batch_size=batch_size, collate_fn= pad_batch)
dev_loader = data.DataLoader(StructuralDataset(*devxy), batch_size=batch_size, collate_fn= pad_batch)
test_loader = data.DataLoader(StructuralDataset(*testxy), batch_size=batch_size, collate_fn= pad_batch)

In [15]:
from torch import optim
import math

# I recommend you to write a method that can evaluate the UUAS & loss score for the dev (& test) corpus.
# Feel free to alter the signature of this method.
def evaluate_probe(probe, dataloader):
    loss_function =  L1DistanceLoss()
    probe.eval()
    total_loss = 0.0
    total_uuas = 0.0
    amt = 0.0
    for distances, embs, lengths in dataloader:
        embs = embs.to(device)
        distances = distances.to(device)
        lengths = lengths.to(device)
        amt += len(distances)
        outputs = probe(embs)
        loss = loss_function(outputs, distances, lengths)[0]
        total_loss += loss.item()
        for i in range(len(distances)):
            l = lengths[i]
            preds = outputs[i,0:l, 0:l]
            gold = distances[i,0:l, 0:l]
            
            u = tree_utils.calc_uuas(preds, gold)
            if math.isnan(u):
                amt -= 1
            # This if statement is a hack so nans don't get counted
            if u >= 0: total_uuas += u
    
    return total_loss/amt, total_uuas/amt

# Feel free to alter the signature of this method.
def train_structural(probe, dataloader, dev_dataloader,test_loader, epochs=100):
    lr = 1e-5
    batch_size = 128
    
    optimizer = optim.Adam(probe.parameters(), lr=lr)
    #scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5,patience=1)
    loss_function =  L1DistanceLoss()

    for epoch in range(epochs):
        probe.train()
        for distances, embs, lengths in dataloader:
            embs = embs.to(device)
            distances = distances.to(device)
            lengths = lengths.to(device)
            outputs = probe(embs)
            loss = loss_function(outputs, distances, lengths)[0]
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        dev_loss, dev_uuas = evaluate_probe(probe, dev_dataloader)
        print("Epoch", epoch, "Dev loss and uuas", dev_loss, dev_uuas)
        # Using a scheduler is up to you, and might require some hyper param fine-tuning
        #scheduler.step(dev_loss)

    test_loss, test_uuas = evaluate_probe(probe, test_loader)
    print("Test loss, uuas", test_loss, test_uuas)

emb_dim = 768
rank = 64
probe = StructuralProbe(emb_dim, rank).to(device)
print(probe)
train_structural(probe, train_loader, dev_loader, test_loader, epochs=10)

StructuralProbe()
Epoch 0 Dev loss and uuas 4.185664472908809 0.10515590591423847
Epoch 1 Dev loss and uuas 4.180507002205684 0.10515590591423847
Epoch 2 Dev loss and uuas 4.175355582401671 0.10515590591423847
Epoch 3 Dev loss and uuas 4.170209687331627 0.10515590591423847
Epoch 4 Dev loss and uuas 4.165070106243265 0.10515590591423847
Epoch 5 Dev loss and uuas 4.159934997558594 0.10515590591423847
Epoch 6 Dev loss and uuas 4.154806202855603 0.10515590591423847
Epoch 7 Dev loss and uuas 4.149684248299434 0.10515590591423847
Epoch 8 Dev loss and uuas 4.144567292312096 0.10515590591423847
Epoch 9 Dev loss and uuas 4.139457439554149 0.10515590591423847
Test loss, uuas 4.595877583821615 0.1625274179275489


# POS tag control task

In this section we will analyse the accuracy scores that we achieve on the POS tag control task. This allows us to determine the level of selectivity. In order to do so, we will sample values for a new control_train_y (instead of the regular train_y) which is randomly sampled from a distribution (dist) that is similar to the original train_y.

In [32]:
from controltasks import *

In [33]:
# Retrieve the control task y's
control_y_train, control_dict_train = control_y(train_x, train_y, flatten_train, dist)

In [34]:
control_y_test = dev_test_y(control_dict_train, flatten_test, dist)
control_y_dev  = dev_test_y(control_dict_train, flatten_dev , dist)

In [35]:
# Get dataloaders with control task y
control_train_loader = data.DataLoader(POSDataset(train_x, control_y_train), batch_size=16)
control_dev_loader = data.DataLoader(POSDataset(dev_x, control_y_dev), batch_size=16)
control_test_loader = data.DataLoader(POSDataset(test_x, control_y_test), batch_size=16)

In [36]:
# Train the model on control task
model = POSProbe(768, len(train_y.unique())).to(device)
train(model, control_train_loader, control_dev_loader, 10)
print("Test accuracy", eval_given_dataloader(control_test_loader, model))

Epoch 0 accuracy 0.16941529235382308 0.23782234957020057
Epoch 1 accuracy 0.3583208395802099 0.3223495702005731
Epoch 2 accuracy 0.4917541229385307 0.33954154727793695
Epoch 3 accuracy 0.5637181409295352 0.3424068767908309
Epoch 4 accuracy 0.6206896551724138 0.34813753581661894
Epoch 5 accuracy 0.664167916041979 0.3495702005730659
Epoch 6 accuracy 0.7181409295352323 0.35816618911174786
Epoch 7 accuracy 0.7661169415292354 0.35816618911174786
Epoch 8 accuracy 0.7886056971514243 0.3624641833810888
Epoch 9 accuracy 0.8185907046476761 0.3638968481375358
Test accuracy 0.3473053892215569
